# Capítulo 7: Ensemble Learning e Florestas Aleatórias

## Exercícios

### 1. Se você treinou cinco modelos diferentes com exatamente os mesmos dados de treinamento e todos conseguem uma precisão de 95%, existe alguma chance de você combinar esses modelos para obter melhores resultados? Em caso afirmativo, como? Se não, por quê?

Sim, existe uma chance de melhorar os resultados ao combinar esses cinco modelos, mesmo que todos tenham alcançado uma precisão de 95% nos mesmos dados de treinamento. Esse processo de combinar modelos é conhecido como ensembling e pode ser feito utilizando métodos como bagging, boosting, averaging e stacking ou voting.

### 2. Qual a diferença entre os classificadores de votação hard e soft?


Os classificadores de votação **hard** e **soft** são duas abordagens diferentes para combinar as previsões de múltiplos modelos em um ensemble.

### Votação Hard (Hard Voting)

Na votação hard, cada modelo individual faz uma previsão sobre a classe mais provável para uma amostra. A classe que receber mais votos (ou seja, que for predita pela maioria dos modelos) é escolhida como a previsão final.

- **Funcionamento:** Cada modelo emite uma previsão de classe (por exemplo, `Classe A` ou `Classe B`), e a classe mais votada entre os modelos é selecionada como a saída final.
- **Exemplo:** Se você tiver três modelos, e dois modelos predizem `Classe A` e um modelo prediz `Classe B`, a previsão final será `Classe A`, pois foi a mais votada.
- **Uso:** A votação hard é simples e direta, mas não leva em conta a confiança dos modelos nas suas previsões.

### Votação Soft (Soft Voting)

Na votação soft, cada modelo fornece as probabilidades associadas a cada classe em vez de apenas a classe prevista. As probabilidades de todas as classes são somadas ou ponderadas, e a classe com a maior soma de probabilidades é escolhida como a previsão final.

- **Funcionamento:** Cada modelo retorna uma distribuição de probabilidades sobre todas as classes. Essas probabilidades são somadas (ou ponderadas se você quiser atribuir pesos diferentes aos modelos) para cada classe, e a classe com a maior probabilidade agregada é selecionada como a previsão final.
- **Exemplo:** Se três modelos geram as seguintes probabilidades para `Classe A`: `0.6`, `0.7` e `0.4`, a soma das probabilidades para `Classe A` será `0.6 + 0.7 + 0.4 = 1.7`. A classe com a maior soma será a escolhida.
- **Uso:** A votação soft geralmente é preferida quando os modelos conseguem fornecer as probabilidades, pois essa abordagem considera a confiança de cada modelo em suas previsões. Isso pode levar a uma melhor performance geral do ensemble.

### Comparação e Aplicações

- **Votação Hard:**
  - Simples de implementar.
  - Útil quando os modelos não fornecem probabilidades ou quando você deseja uma decisão simples e majoritária.
  
- **Votação Soft:**
  - Pode ser mais precisa, pois leva em conta a confiança dos modelos nas suas previsões.
  - Requer que os modelos forneçam probabilidades de classe.

Em geral, a votação soft tende a ter um desempenho melhor, especialmente em casos onde os modelos individuais variam em termos de confiança ou performance.

### 3. É possível acelerar o treinamento de um bagging ensemble distribuindo-o por vários servidores? E com pasting ensembles, boosting ensembles, florestas aleatórias, ou stacking ensembles?

Sim, é possível acelerar o treinamento de ensembles como bagging, pasting e florestas aleatórias distribuindo a carga de trabalho por vários servidores, mas para stacking e boosting, em geral, isso não é possível pelo menos completamente, pois nesse modelos há etapas de treinamento sequencial, o que impede a paralelização.

### 4. Qual é o benefício da avaliação out-of-bag?

A avaliação OOB oferece uma maneira eficiente e confiável de estimar a performance do modelo sem a necessidade de um conjunto de validação separado, permitindo o uso total dos dados disponíveis e economizando recursos computacionais. Ela também oferece uma estimativa de importância de variáveis, tornando-a uma ferramenta valiosa na construção de ensembles baseados em bagging.

### 5. O que torna as Árvores-Extras mais aleatórias do que as Florestas Aleatórias comuns? Como esta aleatoriedade extra pode ajudar? As Árvores-Extras são mais lentas ou mais rápidas do que as Florestas Aleatórias regulares?

As Árvores-Extras introduzem mais aleatoriedade ao selecionar os pontos de divisão aleatoriamente e ao não utilizarem bootstrap, tornando-as mais rápidas de treinar do que as Florestas Aleatórias tradicionais. Essa aleatoriedade extra pode ajudar a reduzir o overfitting e melhorar a capacidade de generalização do modelo em certos contextos, especialmente em dados ruidosos ou complexos.

### 6. Se o seu ensemble AdaBoost se subajusta aos dados de treinamento, quais hiperparâmetros você deve ajustar e como?

Se o ensemble AdaBoost estiver se subajustado ao conjunto de treinamento, pode-se tentar aumentar o número de estimadores, reduzir os hiperparâmetros que regularizam o estimador base ou até aumentar a taxa de aprendizzado.

### 7. Se o seu ensemble Gradient Boosting se sobreajusta ao conjunto de treinamento, você deve aumentar ou diminuir a taxa de aprendizado?

Para reduzir o overfitting em um modelo de Gradient Boosting, diminuir a taxa de aprendizado é uma das principais estratégias. Isso ajuda a suavizar a contribuição de cada árvore, tornando o modelo final menos propenso a capturar ruído e mais capaz de generalizar para novos dados.

### 8. Carregue os dados MNIST (introduzido no Capítulo 3) e o divida em um conjunto de treinamento, um conjunto de validação e um conjunto de teste (por exemplo, utilize 40 mil instâncias para treinamento, 10 mil para validação e 10 mil para teste). Em seguida, treine vários classificadores como um classificador Floresta Aleatória, um classificador Árvores-extra e um SVM. Em seguida, utilizando um classificador de votação soft ou hard, tente combiná-los em um ensemble que supere todos no conjunto de validação. Uma vez tendo encontrado o ensemble, teste-o no conjunto de teste. Qual é a melhoria de desempenho em comparação com os classificadores individuais?

**Carregando e separando os dados MNIST**

In [3]:
import numpy as np
from sklearn import datasets

mnist = datasets.fetch_openml('mnist_784', version=1, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:40000]
y_train = y[:40000]
X_test = X[40000:50000]
y_test = y[40000:50000]
X_val = X[50000:60000]
y_val = y[50000:60000]

**Treinando vários classificadores, como uma Floresta Aleatória, uma Árvore-Extra e um SVM**

  - Floresta Aleatória

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia da Floresta Aleatória: {accuracy:.2f}')


Acurácia da Floresta Aleatória: 0.94


  - Árvore-Extra 

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

extra_trees = ExtraTreesClassifier(n_estimators=100, max_depth=10, random_state=42)

extra_trees.fit(X_train, y_train)

y_pred = extra_trees.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo de Árvores-Extras: {accuracy:.2f}')


Acurácia do modelo de Árvores-Extras: 0.93


  - SVM

In [6]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(random_state=42)

linear_svc.fit(X_train, y_train)

y_pred = linear_svc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo SVM linear: {accuracy:.2f}')

Acurácia do modelo SVM linear: 0.90


**Combinando os três classificadores em um ensemble**

In [7]:
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("random_forest", random_forest),
    ("extra_trees", extra_trees),
    ("linear_svc", linear_svc)
]

voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(max_depth=10,
                                                     random_state=42)),
                             ('extra_trees',
                              ExtraTreesClassifier(max_depth=10,
                                                   random_state=42)),
                             ('linear_svc', LinearSVC(random_state=42))])

In [8]:
voting_clf.score(X_val, y_val)

0.9498

A acurácia final obtida não foi tão alta, o que pode ser melhorado trocando o SVM linear por um modelo que tenha uma melhor acurácia, como uma rede neural, ou então escolhendo melhor os hiperparâmetros dos previsores por meio de um gridsearch 

### 9. Execute os classificadores individuais do exercício anterior para fazer previsões no conjunto de validação e crie um novo conjunto de treinamento com as previsões resultantes: cada instância de treinamento é um vetor que contém o conjunto de previsões de todos os seus classificadores para uma imagem e o alvo é a classe da imagem. Parabéns, você acabou de treinar um blender, e, junto com os classificadores, eles formam um stacking ensemble! Agora, avaliaremos o conjunto no conjunto de testes. Para cada imagem no conjunto de teste, faça previsões com todos os seus classificadores, então forneça as previsões ao blender para obter as previsões do ensemble. Como ela se compara ao classificador de votação que você treinou anteriormente?

In [11]:
y_pred_florest = random_forest.predict(X_val)
y_pred_extra_tree = extra_trees.predict(X_val)
y_pred_svm = linear_svc.predict(X_val)

X_train_blender = []

for k in range(10000):
    X_train_blender.append((y_pred_florest[k], y_pred_extra_tree[k], y_pred_svm[k]))

print(X_train_blender)

[(3, 3, 3), (8, 8, 8), (6, 6, 6), (9, 9, 9), (6, 6, 6), (4, 4, 4), (5, 5, 5), (3, 3, 3), (8, 8, 8), (4, 4, 4), (5, 5, 5), (2, 2, 2), (3, 3, 3), (8, 8, 8), (4, 4, 4), (8, 8, 8), (1, 1, 1), (5, 5, 5), (0, 0, 0), (5, 5, 5), (9, 9, 9), (7, 7, 7), (4, 4, 4), (1, 1, 1), (0, 0, 0), (3, 3, 6), (0, 0, 0), (6, 6, 6), (2, 2, 2), (9, 9, 9), (9, 9, 9), (4, 4, 4), (1, 1, 1), (3, 3, 3), (6, 6, 6), (8, 8, 8), (0, 0, 0), (7, 7, 7), (7, 7, 7), (6, 6, 6), (8, 8, 8), (9, 9, 9), (0, 0, 0), (3, 3, 3), (8, 8, 8), (3, 3, 3), (7, 7, 7), (7, 7, 7), (8, 8, 8), (4, 4, 4), (4, 4, 4), (1, 1, 1), (2, 2, 6), (9, 9, 9), (8, 8, 6), (1, 1, 1), (1, 1, 1), (0, 0, 0), (6, 6, 6), (6, 6, 6), (5, 5, 5), (0, 0, 0), (1, 1, 1), (1, 1, 1), (7, 7, 7), (2, 2, 2), (7, 7, 7), (3, 3, 3), (1, 1, 1), (4, 4, 4), (0, 0, 0), (5, 5, 5), (0, 0, 0), (6, 6, 6), (9, 9, 7), (7, 7, 7), (6, 6, 6), (8, 8, 8), (8, 5, 3), (9, 9, 9), (4, 4, 4), (0, 0, 0), (6, 6, 6), (1, 1, 1), (9, 9, 9), (2, 2, 2), (6, 6, 6), (3, 3, 8), (7, 7, 1), (4, 4, 4), (4, 4, 9)

In [12]:
rnd_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_blender.fit(X_train_blender, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [13]:
rnd_blender.oob_score_

0.9488

In [14]:
y_test_florest = random_forest.predict(X_test)
y_test_extra_tree = extra_trees.predict(X_test)
y_test_svm = linear_svc.predict(X_test)

X_test_blender = []

for k in range(10000):
    X_test_blender.append((y_test_florest[k], y_test_extra_tree[k], y_test_svm[k]))
    
y_pred = rnd_blender.predict(X_test_blender)

print(accuracy_score(y_test, y_pred))

0.935


O resultado obtido com o blender foi um pouco abaixo do obtido com o ensemble, o que pode ser contornado escolhendo melhores hirparâmetros da floresta aleatória utilizada para fazer as previsões ou trocando o modelo utilzado no blender. 